In [82]:
# import models
import mplfinance as mpf
from config import api_key
import requests
import pandas as pd
import time
import datetime
from datetime import time
import matplotlib.pyplot as plt
import cufflinks as cf
import os


In [83]:
#input info used to pass through api 
symbol = input("What ticker do you want to search?")
periodtype = input("Period type (day, month, year)")
frequency = input("Frequency of data")
frequencytype = input("Frequency type (minute, monthly, daily)")
period = input("Period of data")
needExtendedHoursData = input('Extended hours data (True or False)')


What ticker do you want to search?QQQ
Period type (day, month, year)year
Frequency of data1
Frequency type (minute, monthly, daily)monthly
Period of data20
Extended hours data (True or False)False


In [84]:
#url for td
url = "https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(symbol)
    
#parameters
key = {'apikey':api_key,
         'periodType':periodtype,
          'frequencyType': frequencytype,
          'frequency': frequency,
          'period': period,
          'needExtendedHoursData': needExtendedHoursData,
         }
#requesting data
content = requests.get(url = url, params=key)

data = content.json()
data

{'candles': [{'open': 25.97,
   'high': 26.55,
   'low': 21.64,
   'close': 23.85,
   'volume': 2668049100,
   'datetime': 1025499600000},
  {'open': 23.74,
   'high': 26.21,
   'low': 21.3,
   'close': 23.49,
   'volume': 2043810216,
   'datetime': 1028178000000},
  {'open': 23.06,
   'high': 24.35,
   'low': 20.49,
   'close': 20.72,
   'volume': 1669047392,
   'datetime': 1030856400000},
  {'open': 20.91,
   'high': 25.04,
   'low': 19.76,
   'close': 24.55,
   'volume': 2129150120,
   'datetime': 1033448400000},
  {'open': 24.37,
   'high': 28.29,
   'low': 24.14,
   'close': 27.72,
   'volume': 1669954692,
   'datetime': 1036130400000},
  {'open': 28.42,
   'high': 28.79,
   'low': 24.28,
   'close': 24.37,
   'volume': 1397055816,
   'datetime': 1038722400000},
  {'open': 24.72,
   'high': 27.47,
   'low': 24.01,
   'close': 24.44,
   'volume': 1513988020,
   'datetime': 1041400800000},
  {'open': 24.52,
   'high': 25.37,
   'low': 23.32,
   'close': 25.16,
   'volume': 127662248

In [85]:
#create arrays for each category
open_tick = []
high_tick = []
low_tick = []
close_tick = []
volume_tick = []
datetime_tick = []
#iterrates through data to append each array with it's value
for i in range(len(data["candles"])):
    open_tick.append(data["candles"][i]["open"])
    high_tick.append(data["candles"][i]["high"])
    low_tick.append(data["candles"][i]["low"])
    close_tick.append(data["candles"][i]["close"])
    volume_tick.append(data["candles"][i]["volume"])
    datetime_tick.append(data["candles"][i]["datetime"])

#converting epoch to datetime       
file = []
for i in range(len(datetime_tick)):
    # divide epoch by 1000 because it is in milliseconds
    test = datetime_tick[i]/1000
    file.append(datetime.datetime.fromtimestamp(test))

#convert to dataframe
theta_df = pd.DataFrame({
            "Open":open_tick,
            "Close":close_tick,
            "High":high_tick,
            "Low":low_tick,
            "Volume":volume_tick,
            "Datetime": file
    })

In [86]:
# set index to Datetime
theta_df = theta_df.set_index("Datetime")

In [87]:
# Calculate % gain for the day 
theta_df["Gain%"] = (theta_df.Close - theta_df.Open)/theta_df.Open * 100

theta_df

,Open,Close,High,Low,Volume,Gain%
Datetime,,,,,,
2002-06-30 22:00:00,25.97,23.85,26.55,21.64,2668049100,-8.163265
2002-07-31 22:00:00,23.74,23.49,26.21,21.30,2043810216,-1.053075
2002-08-31 22:00:00,23.06,20.72,24.35,20.49,1669047392,-10.147441
2002-09-30 22:00:00,20.91,24.55,25.04,19.76,2129150120,17.407939
2002-10-31 22:00:00,24.37,27.72,28.29,24.14,1669954692,13.746410
...,...,...,...,...,...,...
2022-01-31 22:00:00,364.43,346.80,370.10,318.26,1523185568,-4.837692
2022-02-28 22:00:00,345.75,362.54,371.83,317.45,1686866041,4.856110
2022-03-31 22:00:00,362.81,313.25,369.31,312.60,1501958749,-13.660042


In [88]:
# for the stock ticker if a folder does not exist already, one will be created
mypath = f"{symbol}"
if not os.path.isdir(mypath):
   os.makedirs(mypath)

In [89]:
# uploads csv to that folder specifying the frequency and type to identify the csv
upload_file = f"{symbol}/{symbol}_{frequencytype}_{frequency}_{needExtendedHoursData}.csv"
theta_df.to_csv(upload_file, index = True)